In [1]:
import pandas3js as pjs
print('pandasjs version = {}'.format(pjs.__version__))
import jsonextended as ejson
print('jsonextended version = {}'.format(ejson.__version__))
import pandas as pd
import numpy as np
import pythreejs as tjs

pandasjs version = 0.1.4
jsonextended version = 0.1.3.3


In [3]:
data = ejson.json_to_dict(ejson.get_test_path())
ejson.dict_pprint(data, depth=3)

dir1: 
  dir1_1: 
    file1_1: {...}
  file1: 
    initial: {...}
    meta: {...}
    optimised: {...}
    units: {...}
  file2: 
    initial: {...}
    meta: {...}
    optimisation: {...}
    optimised: {...}
    units: {...}
dir2: 
  file1: 
    initial: {...}
    meta: {...}
    optimisation: {...}
    optimised: {...}
    units: {...}
dir3: 


In [4]:
data = ejson.dict_remove_paths(data, ['units'])
energies = ejson.dict_filter_keys(data, ['energy'])
pd.Series(ejson.dict_flatten(energies)).describe()

count       89.000000
mean    -24062.286741
std          0.012361
min     -24062.293964
25%     -24062.293727
50%     -24062.293491
75%     -24062.278072
max     -24062.207939
dtype: float64

In [5]:
energies = ejson.units.apply_unitschema(energies, {'energy':'eV'})
energies = ejson.units.apply_unitschema(energies, {'energy':'kcal'},as_quantity=False)
pd.Series(ejson.dict_flatten(energies)).describe()

count    8.900000e+01
mean    -9.214157e-19
std      4.733481e-25
min     -9.214159e-19
25%     -9.214159e-19
50%     -9.214159e-19
75%     -9.214153e-19
max     -9.214126e-19
dtype: float64

In [6]:
data = ejson.json_to_dict(ejson.get_test_path())
optimisation = ejson.dict_multiindex(data,['dir1','file2','optimisation','steps'])
optsteps = sorted(optimisation.keys(), key=ejson.core._natural_keys)
print(optsteps)

[u'2', u'3', u'4', u'5', u'6', u'7', u'8', u'9', u'10', u'11', u'12', u'13', u'14', u'15', u'16', u'17', u'18', u'19', u'20', u'21', u'22', u'23', u'24', u'25', u'26', u'27', u'28', u'29', u'30', u'31', u'32', u'33', u'34', u'35', u'36', u'37', u'38', u'39', u'40', u'41', u'42', u'43', u'44', u'45', u'46', u'47', u'48', u'49', u'50', u'51', u'52', u'53', u'54', u'55', u'56', u'57', u'58', u'59', u'60', u'61', u'62', u'63', u'64', u'65', u'66', u'67', u'68', u'69', u'70', u'71', u'72', u'73', u'74', u'75', u'76', u'77', u'78', u'79', u'80', u'81', u'82', u'83', u'84', u'85', u'86', u'87', u'88', u'89', u'90']


In [7]:
ejson.dict_pprint(optimisation['2'],
                  depth=None,no_values=True)

crystallographic: 
  geometry: 
    assym:         
    atomic_number: 
    id:            
    label:         
    x/a:           
    y/b:           
    z/c:           
  lattice_parameters: 
    a:     
    alpha: 
    b:     
    beta:  
    c:     
    gamma: 
  volume: 
energy: 
primitive: 
  density: 
  geometry: {...}
  lattice_parameters: {...}
  volume:  


In [8]:
ejson.dict_to_html(optimisation['2'])

In [2]:
reload(pjs)
from scipy.spatial import cKDTree
cKDTree.query_pairs

def format_label(s):
    s = ''.join([i for i in s if not i.isdigit()])
    return s.lower().capitalize()

def change_crystal_opt(data, options):
    
    geotype = 'primitive'#'crystallographic'

    # lattice vectors
    ldict = data[geotype]['lattice_parameters']
    a_vec, b_vec, c_vec = pjs.atom.vectors_from_params(
         *[ldict[s] for s in ('a','b','c','alpha','beta','gamma')])
    
    gdict = data[geotype]['geometry']
    df = pd.DataFrame(gdict)
        
    xyz = (df['x/a'].apply(lambda x:x*a_vec) + 
       df['y/b'].apply(lambda y:y*b_vec) + 
       df['z/c'].apply(lambda z:z*c_vec))
    xyz = np.array(xyz.values.tolist())
    df['x'] = xyz[:,0]
    df['y'] = xyz[:,1]
    df['z'] = xyz[:,2]
    
    df.label = df.label.apply(format_label)
    
    atom_df = pjs.atom.atomic_data()
    df['color'] = df['atomic_number'].map(lambda n: atom_df.loc[n].color)
    df['label_color'] = df['color']
    df['radius'] = df['atomic_number'].map(lambda n: atom_df.loc[n].RCov)
    df['transparency'] = 1
    
    df = df[['id','x','y','z',
             'label','label_color',
             'color','radius']]
    df['transparency'] = 1
    df['otype'] = 'pandas3js.Sphere'
    
    # sub-lattices
    if options.get('show coordination', False):
        for aname, anumber in (['S',16],):#,['Fe',26]):
            r_array = df[df.label==aname][['x','y','z']].values    
            ck = cKDTree(r_array)
            pairs = ck.query_pairs(4)
            for i,j in pairs:
                series = pd.Series({
                    'start':tuple(r_array[i]),'end':tuple(r_array[j]),
                    'color':atom_df.loc[anumber].color,'otype':'pandas3js.Line',
                    'id':df['id'].max()+1,'linewidth':3})
                df = df.append(series,ignore_index=True,)
    
    if options.get('show cell box',False):
        series = pd.Series({'a':tuple(a_vec),'b':tuple(b_vec),'c':tuple(c_vec),
                            'otype':'pandas3js.WireBox','id':-1,'color':'black'})    
        df = df.append(series,ignore_index=True,)

    return df

In [3]:
data = ejson.json_to_dict(ejson.get_test_path(),['dir1','file2','optimisation','steps'])
gui, geometry = pjs.create_gui(change_crystal_opt,data,near=-15,
                                  add_labels=True,view=(10, -10, -10, 10),
                                  otype_column='otype',
                                  add_options={'show coordination':[True, False],
                                              'show cell box':[True,False]})
gui

In [169]:
np.polynomial.legendre.Legendre()

# Example

In [1]:
import pandas as pd
import pandas3js as pjs

data = {'1':{'id':[0,1,2],
             'geometry':{
                 'x':[0,1,2],'y':[0,1,0],'z':[0,0,1]},
              'c1':'blue','c2':'red'},
        '2':{'id':[0,1,2],
             'geometry':{
                 'x':[0,.5,1],'y':[0,0,1],'z':[0,0.5,1]},
              'c1':'blue','c2':'red'}}

def change_func(cdata,options):
    df = pd.DataFrame(cdata['geometry'])
    ctype = options.get('color','c1')
    df['id'] = cdata['id']
    df['color'] = cdata[ctype]
    df['radius'] = 0.5
    df['label'] = 'A'
    df['otype'] = 'pandas3js.Sphere'
    return df

gui, gcollect = pjs.create_config_gui(data,change_func,
                    height=200,width=200,
                    view=(3,-3,-3,3),otype_column='otype',
                    add_options={'color':['c1','c2']})

In [4]:
gui

In [5]:
trait_df = gcollect.trait_df()
trait_df

,color,id,label,label_color,label_transparency,label_visible,otype,radius,transparency,visible,x,y,z
0,blue,0,A,red,1.0,False,pandas3js.models.idobject.Sphere,0.5,1.0,True,0.0,0.0,0.0
1,blue,1,A,red,1.0,False,pandas3js.models.idobject.Sphere,0.5,1.0,True,0.5,0.0,0.5
2,blue,2,A,red,1.0,False,pandas3js.models.idobject.Sphere,0.5,1.0,True,1.0,1.0,1.0


In [6]:
trait_df.transparency = 0.1
gcollect.change_by_df(trait_df,
                      otype_column='otype')

In [4]:
render = gui.children[1]


In [80]:
render.camera.position = [0.15112475372064813, 0.41804944513679193, .9]

In [59]:
render = gui.children[1]
alpha = np.radians(50)
print(render.camera.quaternion)
render.camera.quaternion_from_rotation([ 
                np.cos(alpha), -np.sin(alpha), 0, 
                np.sin(alpha), np.cos(alpha),  0, 
                0,             0,              1
            ])
print(render.camera.quaternion)


[0.04293297400510096, 0.41105752390311934, -0.01938498105751881, 0.9103915029200877]
[0.0, 0.0, -0.4226182617406994, 0.90630778703665]


In [82]:
control = render.controls[0]

# Implementation of Spherical Harmonics

In [ ]:
# from scipy.special

cimport sf_error

cdef extern from "specfun_wrappers.h":
    double pmv_wrap(double, double, double) nogil

cdef extern from "c_misc/misc.h":
    double poch(double x, double m) nogil

from _complexstuff cimport *
from libc.math cimport cos, sqrt, fabs
from libc.stdlib cimport abs

cdef inline double complex sph_harmonic(int m, int n, double theta, double phi) nogil:
    cdef double x, prefactor
    cdef double complex val
    cdef int mp
    x = cos(phi)
    if abs(m) > n :
        sf_error.error("sph_harm", sf_error.ARG, "m should not be greater than n")
        return nan
    if n < 0:
        sf_error.error("sph_harm", sf_error.ARG, "n should not be negative")
        return nan
    if m < 0:
        mp = -m
        prefactor = (-1)**mp * poch(n + mp + 1, -2 * mp)
    else:
        mp = m
    val = pmv_wrap(mp, n, x)
    if  m < 0:
        val *= prefactor
    val *= sqrt((2*n + 1) / 4.0 / pi)
    val *= sqrt(poch(n + m + 1, -2 * m))
    val *= zexp(1j * m * theta)
return val

poch         -- The Pochhammer symbol (rising factorial).
/*
* Pochhammer symbol (a)_m = gamma(a + m) / gamma(a)
*/

#include <Python.h>
#include <numpy/npy_math.h>

#include <math.h>
#include "cephes.h"
#include "misc.h"

static double is_nonpos_int(double x)
{
   return x <= 0 && x == ceil(x) && fabs(x) < 1e13;
}

double poch(double a, double m)
{
   double r;

   r = 1.0;

   /*
    * 1. Reduce magnitude of `m` to |m| < 1 by using recurrence relations.
    *
    * This may end up in over/underflow, but then the function itself either
    * diverges or goes to zero. In case the remainder goes to the opposite
    * direction, we end up returning 0*INF = NAN, which is OK.
    */

   /* Recurse down */
   while (m >= 1.0) {
       if (a + m == 1) {
           break;
       }
       m -= 1.0;
       r *= (a + m);
       if (!npy_isfinite(r) || r == 0) {
           break;
       }
   }

   /* Recurse up */
   while (m <= -1.0) {
       if (a + m == 0) {
           break;
       }
       r /= (a + m);
       m += 1.0;
       if (!npy_isfinite(r) || r == 0) {
           break;
       }
   }

   /*
    * 2. Evaluate function with reduced `m`
    *
    * Now either `m` is not big, or the `r` product has over/underflown.
    * If so, the function itself does similarly.
    */

   if (m == 0) {
       /* Easy case */
       return r;
   }
   else if (a > 1e4 && fabs(m) <= 1) {
       /* Avoid loss of precision */
       return r * pow(a, m) * (
           1
           + m*(m-1)/(2*a)
           + m*(m-1)*(m-2)*(3*m-1)/(24*a*a)
           + m*m*(m-1)*(m-1)*(m-2)*(m-3)/(48*a*a*a)
           );
   }

   /* Check for infinity */
   if (is_nonpos_int(a + m) && !is_nonpos_int(a) && a + m != m) {
       return NPY_INFINITY;
   }

   /* Check for zero */
   if (!is_nonpos_int(a + m) && is_nonpos_int(a)) {
       return 0;
   }

   return r * exp(lgam(a + m) - lgam(a)) * gammasgn(a + m) * gammasgn(a);
}


double pmv_wrap(double m, double v, double x){
  int int_m;
  double out;

  if (m != floor(m)) return NPY_NAN;
  int_m = (int ) m;
  F_FUNC(lpmv,LPMV)(&v, &int_m, &x, &out);
  CONVINF("pmv", out);
  return out;
}
#define CONVINF(func, x)                                                \
    do {                                                                \
        if ((x) == 1.0e300) {                                           \
            sf_error(func, SF_ERROR_OVERFLOW, NULL);                    \
            (x)=NPY_INFINITY;                                           \
        }                                                               \
        if ((x)==-1.0e300) {                                            \
            sf_error(func, SF_ERROR_OVERFLOW, NULL);                    \
            (x)=-NPY_INFINITY;                                          \
        }                                                               \
    } while (0)


In [ ]:
import math as Math

m=1
n=2
origu = .1
origv = .1

u = 2*Math.pi*origu
v = 2*Math.pi*origv

x = Math.cos(v)

if abs(m) > n:
    print('error')
if n < 0:
    print('error')
if m < 0:
    mp = -m
    print('todo')
    #prefactor = (-1)**mp * poch(n + mp + 1, -2 * mp)
else:
    mp = m    

def naive_lpmv(m, v, x):
    poly = np.polynomial.legendre.Legendre([0]*v + [1])
    return poly.deriv(m)(x) * (1-x*x)**(m/2) * (-1)**m

val = naive_lpmv(mp,n,x)

if  m < 0:
    print('todo')
    #val *= prefactor

val *= Math.sqrt((2*n + 1) / 4.0 / Math.pi)
val

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.special import sph_harm

phi,theta = np.mgrid[0:2*np.pi:200j, 0:np.pi:100j]

# The Cartesian coordinates of the unit sphere
x = np.sin(theta) * np.cos(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(theta)


l=2

for m in range(-l,l+1):
    
    print l,m

    fig = plt.figure(figsize=plt.figaspect(1.))
    ax = fig.add_subplot(111, projection='3d')
    # Calculate the spherical harmonic Y(l,m) and normalize to [0,1]
    fcolors = np.abs(sph_harm(m, l, phi, theta))
    fmax, fmin = fcolors.max(), fcolors.min()
    fcolors = (fcolors - fmin)/(fmax - fmin)

    # Set the aspect ratio to 1 so our sphere looks spherical
    ax.plot_surface(fcolors*x, fcolors*y, fcolors*z,  rstride=1, cstride=1, facecolors=cm.jet(fcolors))
    # Turn off the axis planes
    ax.set_axis_off()
    plt.show()

In [ ]:
f = """
function f(origu,origv) {
    // scale u and v to the ranges I want: [0, 2*pi]
    var u = 2*Math.PI*origu;
    var v = 2*Math.PI*origv;
    
    var x = Y*Math.sin(u);
    var y = Y*Math.cos(v);
    var z = Y*Math.cos(u+v);

    return new THREE.Vector3(x,y,z)
}
"""
surf_g = tjs.ParametricGeometry(func=f);

surf = tjs.Mesh(geometry=surf_g, material=tjs.LambertMaterial(color='green', side='FrontSide'))
surf2 = tjs.Mesh(geometry=surf_g, material=tjs.LambertMaterial(color='yellow', side='BackSide'))
scene = tjs.Scene(children=[surf, surf2, tjs.AmbientLight(color='#777777')])
c = tjs.PerspectiveCamera(position=[5, 5, 3], up=[0, 0, 1],
                      children=[tjs.DirectionalLight(color='white',
                                                 position=[3, 5, 1],
                                                 intensity=0.6)])
renderer = tjs.Renderer(camera=c, scene=scene, controls=[tjs.OrbitControls(controlling=c)])
renderer

https://github.com/polarch/Spherical-Harmonic-Transform-JS

In [5]:
f3 = """
function f(origu,origv) {
    // factorial compute factorial
    var factorial = function (n) {
        if (n === 0) return 1;
        return n * factorial(n - 1);
    }

    // recurseLegendrePoly computes associated Legendre functions recursively
    var recurseLegendrePoly = function (n, x, Pnm_minus1, Pnm_minus2) {

        var Pnm = new Array(n+1);
        switch(n) {
            case 1:
                var x2 = x*x;
                var P10 = x;
                var P11 = Math.sqrt(1-x2);
                Pnm[0] = P10;
                Pnm[1] = P11;
                break;
            case 2:
                var x2 = x*x;
                var P20 = 3*x2;
                P20 = P20*1;
                P20 = P20/2;
                var P21 = 1-x2;
                P21 = Math.sqrt(P21);
                P21 = 3*P21;
                P21 = P21*x;
                var P22 = 1-x2;
                P22 = 3*P22;
                Pnm[0] = P20;
                Pnm[1] = P21;
                Pnm[2] = P22;
                break;
            default:
                var x2 = x*x;
                var one_min_x2 = 1-x2;
                // last term m=n
                var k = 2*n-1;
                var dfact_k = 1;
                if ((k % 2) == 0) {
                    for (var kk=1; kk<k/2+1; kk++) dfact_k = dfact_k*2*kk;
                }
                else {
                    for (var kk=1; kk<(k+1)/2+1; kk++) dfact_k = dfact_k*(2*kk-1);
                }
                Pnm[n] = numeric.mul(dfact_k, Math.pow(one_min_x2, n/2));
                // before last term
                Pnm[n-1] = (2*n-1)*x*Pnm_minus1[n-1]; // P_{n(n-1)} = (2*n-1)*x*P_{(n-1)(n-1)}
                // three term recursence for the rest
                for (var m=0; m<n-1; m++) {
                    var temp1 = (2*n-1)*x*Pnm_minus1[m];
                    var temp2 = (n+m-1)*Pnm_minus2[m];
                    Pnm[m] = (temp1-temp2)/(n-m); // P_l = ( (2l-1)xP_(l-1) - (l+m-1)P_(l-2) )/(l-m)
                }
        }
        return Pnm;
    }

    // computeRealSH computes real spherical harmonics up to order N
    var computeRealSH = function (N, azi, elev) {

        var factorials = new Array(2*N+1);
        // precompute factorials
        for (var i = 0; i < 2*N+1; i++) factorials[i] = factorial(i);
        var Ndirs = azi.length;
        var Nsh = (N+1)*(N+1);
        var leg_n_minus1 = 0;
        var leg_n_minus2 = 0;
        var leg_n;
        var sinel = Math.sin(elev);
        var index_n = 0;
        var Y_N = new Array(Nsh);
        var Nn0, Nnm;
        var cosmazi, sinmazi;


        for (var n = 0; n<N+1; n++) {
            if (n==0) {
                var temp0 = new Array(azi.length);
                temp0.fill(1);
                Y_N[n] = temp0;
                index_n = 1;
            }
            else {
                leg_n = recurseLegendrePoly(n, sinel, leg_n_minus1, leg_n_minus2);
                Nn0 = Math.sqrt(2*n+1);
                for (var m = 0; m<n+1; m++) {
                    if (m==0) Y_N[index_n+n] = Nn0*leg_n[m];
                    else {
                        Nnm = Nn0*Math.sqrt( 2 * factorials[n-m]/factorials[n+m] );
                        cosmazi = Math.cos(m*azi);
                        sinmazi = Math.sin(m*azi);
                        Y_N[index_n+n-m] = Nnm * leg_n[m] * sinmazi;
                        Y_N[index_n+n+m] = Nnm * leg_n[m] * cosmazi;
                    }
                }
                index_n = index_n+2*n+1;
            }
            leg_n_minus2 = leg_n_minus1;
            leg_n_minus1 = leg_n;
        }

        return Y_N;
    }

    // scale u and v to the ranges I want: [0, 2*pi]
    var u = 1*Math.PI*origu;
    var v = 2*Math.PI*origv;
    
    var x = Math.sin(u)*Math.cos(v);
    var y = Math.sin(u)*Math.sin(v);
    var z = Math.cos(u);

    var Y = Math.abs(computeRealSH(2,u,v)[6]);
        
    return new THREE.Vector3(Y*x,Y*y,Y*z)
}
"""

In [ ]:
surf_g = tjs.ParametricGeometry(func=f3);

surf = tjs.Mesh(geometry=surf_g, material=tjs.LambertMaterial(color='green', side='FrontSide'))
#surf2 = tjs.Mesh(geometry=surf_g, material=tjs.LambertMaterial(color='yellow', side='BackSide'))
scene = tjs.Scene(children=[surf, tjs.AmbientLight(color='#777777')])
c = tjs.PerspectiveCamera(position=[5, 5, 3], up=[0, 0, 1],
                      children=[tjs.DirectionalLight(color='white',
                                                 position=[3, 5, 1],
                                                 intensity=0.6)])
renderer = tjs.Renderer(camera=c, scene=scene, controls=[tjs.OrbitControls(controlling=c)])
renderer